In [1]:
suppressMessages(library(dplyr))
suppressMessages(library(ggplot2))
suppressMessages(library(StatsBombR))

Warning message:
“replacing previous import ‘foreach::accumulate’ by ‘purrr::accumulate’ when loading ‘StatsBombR’”Warning message:
“replacing previous import ‘jsonlite::flatten’ by ‘purrr::flatten’ when loading ‘StatsBombR’”Warning message:
“replacing previous import ‘foreach::when’ by ‘purrr::when’ when loading ‘StatsBombR’”

In [8]:
event_data <- StatsBombFreeEvents()

[1] "Whilst we are keen to share data and facilitate research, we also urge you to be responsible with the data. Please register your details on https://www.statsbomb.com/resource-centre and read our User Agreement carefully."
[1] "Whilst we are keen to share data and facilitate research, we also urge you to be responsible with the data. Please register your details on https://www.statsbomb.com/resource-centre and read our User Agreement carefully."
[1] "Whilst we are keen to share data and facilitate research, we also urge you to be responsible with the data. Please register your details on https://www.statsbomb.com/resource-centre and read our User Agreement carefully."


In [11]:
passes_unclean <- event_data %>% filter(type.name == "Pass") %>% filter(pass.outcome.name %in% c(NA, "Incomplete", "Out"))
passes_unclean <- passes_unclean %>% filter(is.na(pass.type.name))
passes_unclean$pass.outcome <- ifelse(passes_unclean$pass.outcome.name %in% c("Incomplete", "Out"), 0, 1)
passes_clean <- passes_unclean %>% select(player.id, duration, pass.length, pass.angle, pass.height.name, pass.body_part.name, pass.outcome)

In [12]:
head(passes_clean)

player.id,duration,pass.length,pass.angle,pass.height.name,pass.body_part.name,pass.outcome
4659,1.640,31.764761,-1.0789871,Low Pass,Right Foot,1
4654,1.506,17.720045,-0.2860514,High Pass,Right Foot,1
4657,3.453,51.351727,0.1171087,High Pass,Right Foot,1
4657,0.560,2.236068,2.0344439,Ground Pass,Right Foot,0
4635,2.080,21.377558,-1.3825748,Ground Pass,Left Foot,1
4651,0.760,13.601471,2.1995926,Ground Pass,Left Foot,1


In [14]:
passes_clean %>% group_by(pass.height.name) %>% summarise(mean(pass.outcome))

pass.height.name,mean(pass.outcome)
Ground Pass,0.8534262
High Pass,0.3749704
Low Pass,0.5357350


In [21]:
model <- glm(pass.outcome ~ duration + duration^2 + pass.length + pass.length^2 + pass.height.name ,data = passes_clean, family=binomial())

summary(model)

passes_clean$pred <- predict(model, passes_clean, type = "response")

head(passes_clean)


Call:
glm(formula = pass.outcome ~ duration + duration^2 + pass.length + 
    pass.length^2 + pass.height.name, family = binomial(), data = passes_clean)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2567  -0.8928   0.5535   0.5793   1.7013  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                1.784053   0.035627   50.08  < 2e-16 ***
duration                  -0.185486   0.024999   -7.42 1.17e-13 ***
pass.length                0.014437   0.001959    7.37 1.70e-13 ***
pass.height.nameHigh Pass -2.351865   0.046816  -50.24  < 2e-16 ***
pass.height.nameLow Pass  -1.627298   0.053807  -30.24  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 24607  on 21168  degrees of freedom
Residual deviance: 20591  on 21164  degrees of freedom
AIC: 20601

Number of Fisher Scoring iterations: 4


player.id,duration,pass.length,pass.angle,pass.height.name,pass.body_part.name,pass.outcome,pred
4659,1.640,31.764761,-1.0789871,Low Pass,Right Foot,1,0.5771672
4654,1.506,17.720045,-0.2860514,High Pass,Right Foot,1,0.3563310
4657,3.453,51.351727,0.1171087,High Pass,Right Foot,1,0.3853467
4657,0.560,2.236068,2.0344439,Ground Pass,Right Foot,0,0.8471555
4635,2.080,21.377558,-1.3825748,Ground Pass,Left Foot,1,0.8464306
4651,0.760,13.601471,2.1995926,Ground Pass,Left Foot,1,0.8628839


In [22]:
passes_clean %>% filter(pass.height.name == "High Pass") %>% arrange(pred)

player.id,duration,pass.length,pass.angle,pass.height.name,pass.body_part.name,pass.outcome,pred
5055,6.066,18.384777,-0.78539820,High Pass,Right Foot,0,0.1934814
4976,7.586,39.824615,-1.07345370,High Pass,Right Foot,0,0.1978239
7182,8.281,56.089214,0.19739556,High Pass,Right Foot,0,0.2151686
5000,4.096,2.828427,0.78539820,High Pass,Left Foot,0,0.2164069
4972,4.147,7.810250,-0.87605804,High Pass,Head,0,0.2271867
5067,4.973,19.924858,1.26491750,High Pass,NA,0,0.2310176
5055,4.355,12.000000,1.57079640,High Pass,Left Foot,0,0.2310561
5058,6.456,40.607880,0.66320300,High Pass,Right Foot,1,0.2352243
5071,7.573,55.659680,0.31038740,High Pass,Keeper Arm,0,0.2370496
5034,3.640,5.830952,-1.03037680,High Pass,NA,0,0.2388855
